### kernel density estimation and kernel regression for prediction
This notebook is my first attempt to write down the ddiff and dddiff models that I have been working on

mixed recursive/iterative approach (rather than vectorized approach that is harder to generalize
- create separate submodules for KDE and KDreg
    - develop a system for storing differenced but unweighted data and the appropriate mask to lessen memory usage. 
        -1's or 0's depending on i/j/k/l/.... values
    - figure out the deepest level requested by the user and levels that are less deep are just slices of the deeper model. 
        -scout out the tree to build the differenced dataset. 
          
        
 plot and compare
on synthetic data, 1, 2, 3+ mixed distributions
1d - Ndiff vs gaussian kernel vs kernel_tunneling
2d -  


multidimensional x problem
e.g., parameter treatment
"product kernel approach" vs l2 "el two" (radial basis?)distance 

In [1]:
import numpy as np

### generate a simple linear dataset y=xb+e

In [2]:
import data_gen as dg
data1=dg.data_gen(data_shape=(20,2),seed=0)
y=data1.y
x=data1.x[:,1:] #drop first column of x (all 1's)
e=data1.e
n,p=x.shape

### create modeldict
- 'Ndiff_type': refers to the mathematical form of Ndiff
 - 'product'-Ndiff1 multiplied by Ndiff2
 - 'recursive'-Ndiff1's bw is Ndiff 2
- max_bw_Ndiff: is the depth of Ndiffs applied in estimating the bandwidth.
- 'normalize_Ndiffwtsum':
 - 'across' means sum across own level of kernelized-Ndiffs and divide by that sum (CDF approach)
 - 'own_n' means for (n+k)diff where n+K=max_bw_Ndiff
- kern_grid: 
 - no means smooth the original data
 - int means form a grid across all the variables for xin, yxin with int evenly spaced 
 observations (could make flexible than uniform distribution)
- Ndiff_bw_kern:
 - rbfkern means use the radial basis function kernel
 - 'product' means use product kernel like as in liu and yang eq1. 
- 'regression_model':
 - 'NW' means use nadaraya-watson kernel regression
 - 'full_logit' means local logit with all variables entering linearly
 - 'rbf_logit' means local logit with 1 parameters: scaled l2 norm centered on zero 
 (globally or by i?). Is this a new idea?
- outer_x_bw_form: if x is separated into blocks of rvars that are combined within a block using rbf kernel, 
  - 'one_for_all' - use same hx bw for all blocks
  - 'one_per_block' - each block or rvars gets an hx
- kern_grid
  - if int, then create int evnely spaced values from -3 to 3 (standard normal middle ~99%)
  - 'no' means use original data, which is useful for calibrating hyper parameters
- product_kern_norm: when multiplying kernels across random variables, this parameter determines if each random 
variable has its kernels normalized before the product or not
  - 'self' means each random variable has its kernels divided by the sum of kernels 
  across that random variable (as if creating a cdf)
  - 'none' means no normalization prior to taking products across rvar kernels
- hyper_param_form_dict is a nested dictionary
  - Ndiff_exponent is the exponent wrapped around typically a sum of kernels for each Ndiff level 
after level 1 (i.e., (i-j) or centered, obvious/conventional level.)
  - p_bandscale is the parameter specific to each variable (each x in X) used for prediction (y)
  - Ndiff_depth_bw is used as the kernel's bandwidth (h) at each level of Ndiff including at level 1 
  - outer_x_bw vanilla bandwidth for the rbf or product kernel
  - outer_y_bw vanilla bandwidth for y

In [3]:
modeldict1={
    'Ndiff_type':'product',
    'max_bw_Ndiff':2,
    'normalize_Ndiffwtsum':'own_n',
    'kern_grid':'no',
    'outer_kern':'gaussian',
    'Ndiff_bw_kern':'rbfkern',
    'outer_x_bw_form':'one_for_all',
    'regression_model':'NW',
    'product_kern_norm':'none',
    'hyper_param_form_dict':{
        'Ndiff_exponent':'free',
        'p_bandscale':'non-neg',
        'Ndiff_depth_bw':'non-neg',
        'outer_x_bw':'non-neg',
        'outer_y_bw':'non-neg'
        }
    }
#set starting or fixed values for hyper parameters
hyper_paramdict1={
    'Ndiff_exponent':.005*np.ones([p-1,]),
    'p_bandscale':np.ones([p,]),
    'outer_x_bw':np.array([0.5,]),
    'outer_y_bw':np.array([0.5,]),
    'Ndiff_depth_bw':np.ones([p,])*0.15
    }
#assert len(hyper_paramdict1['Ndiff_exponent'])==modeldict1['max_bw_Ndiff'],'Ndiff_exponent does not match ' \
#                                                                            'depth of max_bw_Ndiff'
assert len(hyper_paramdict1['Ndiff_depth_bw'])==modeldict1['max_bw_Ndiff'],'Ndiff_depth_bw does not match ' \
                                                                             'depth of max_bw_Ndiff'
#create optimization dictionary
optimizedict1={
    'method':'Nelder-Mead',
    'hyper_param_dict':hyper_paramdict1,
    'model_dict':modeldict1
    }

#-----------------Calibrate/Optimize--------

In [4]:
import mykern as mk
optimized_Ndiff_kernel=mk.optimize_free_params(y,x,optimizedict1)

opitmization starting
excess numerator mask count 780
excess denom mask count 0
gkernh maskcount 780 kernshape (20, 20, 20) maskeddata_maskcount 780 maskeddata_shape (20, 20, 20) hmaskcount 0
excess numerator mask count 780
excess denom mask count 0
gkernh maskcount 780 kernshape (20, 20, 20) maskeddata_maskcount 780 maskeddata_shape (20, 20, 20) hmaskcount 0
20 xbw masked count 0 xonediffs mask count
40 yx_bw_endstack masked count 0 yonediffs mask count
excess numerator mask count 0
excess denom mask count 0
gkernh maskcount 20 kernshape (20, 20) maskeddata_maskcount 0 maskeddata_shape (20, 20) hmaskcount 20
20 alkerns masked count with shape (20, 20)
excess numerator mask count 0
excess denom mask count 0
gkernh maskcount 40 kernshape (20, 20, 2) maskeddata_maskcount 0 maskeddata_shape (20, 20, 2) hmaskcount 40
40 alkerns masked count with shape (20, 20, 2)
20 are masked in cdfnorm_prob_yx of shape: (20, 20)
20 are masked in cdfnorm_prob_x of shape: (20, 20)
mse: 383.73881416406454
0

In [5]:
print(optimized_Ndiff_kernel.mse)
import myreg
just_linreg=myreg.reg(y,x)
just_linreg.myreg()
print('linreg_mse:',just_linreg.mse)

 final_simplex: (array([[ 0.00321122,  0.8469173 , -0.29390413,  0.35257914, -0.06770112,
        -0.00905423,  1.66152122],
       [ 0.00321113,  0.84690934, -0.29394424,  0.35258601, -0.06770806,
        -0.00906104,  1.66157583],
       [ 0.00321113,  0.8469165 , -0.29396039,  0.35258897, -0.06771018,
        -0.00906681,  1.66157967],
       [ 0.00321115,  0.84695067, -0.29390334,  0.35257473, -0.06769624,
        -0.00903857,  1.66152189],
       [ 0.00321121,  0.84691179, -0.29388643,  0.35257434, -0.06769653,
        -0.00902054,  1.66151084],
       [ 0.00321105,  0.84692465, -0.29397369,  0.35258795, -0.06770981,
        -0.00906106,  1.66158385],
       [ 0.00321126,  0.84690229, -0.29394728,  0.35258493, -0.06771178,
        -0.00906874,  1.66160546],
       [ 0.00321111,  0.84690916, -0.2938431 ,  0.35257532, -0.06769629,
        -0.00901789,  1.66150668]]), array([0.00359907, 0.0035991 , 0.00359913, 0.00359914, 0.00359918,
       0.00359919, 0.00359922, 0.00359925]))
     

In [6]:
from bokeh.io import show, output_notebook#,curdoc,save, output_file
from bokeh.plotting import figure
#from bokeh.models import ColumnDataSource, Range1d, BBoxTileSource
#from bokeh.layouts import row
output_notebook()

Loading BokehJS ...

In [7]:
xgrid=np.linspace(1,n,n)
yhat=optimized_Ndiff_kernel.yhat
p=figure(title='y and yhat', plot_width=900, plot_height=500)
p.xaxis.axis_label = 'observations'
p.scatter(xgrid,y,size=5,color='blue',alpha=0.6,legend='y')
#p.line(xgrid,precip,color='blue',alpha=0.6,legend='precipitation')
p.scatter(xgrid,yhat,size=5,color='green',legend='yhat')
#p.line(xgrid,gageht,color='green',legend='gage height')
p.legend.location = "top_left"
p.yaxis.visible=False
show(p)

In [8]:
yhat.shape

(20,)

In [9]:
print(yhat)

[4.941423864992844 -19.949963942034515 -12.671445275107454
 -3.79311064937101 2.5715109075737352 -8.110713512928724
 -8.074496802051803 -14.929603808625014 -16.238519860406356
 8.834392367095834 -17.079048791457602 -28.978830184757346
 -8.713309037335216 -0.7934020403853967 -4.472093065195821
 -18.98061137277633 -5.098453901240458 -2.6942126116844083
 -8.690766563907722 -14.656277407223893]


In [10]:
print(y)

[  4.92244799 -19.89925284 -12.63540523  -3.83886583   2.52365908
  -8.11852344  -8.08267142 -14.87813497 -16.1817548    8.91476488
 -17.02067093 -29.15991711  -8.71502091  -0.85164944  -4.51302292
 -18.92477785  -5.13444238  -2.74638932  -8.69270722 -14.60629253]


[[ 0.  -4.5 -9. ]
 [ 1.  -3.5 -8. ]
 [ 2.  -2.5 -7. ]
 [ 3.  -1.5 -6. ]
 [ 4.  -0.5 -5. ]
 [ 5.   0.5 -4. ]
 [ 6.   1.5 -3. ]
 [ 7.   2.5 -2. ]
 [ 8.   3.5 -1. ]
 [ 9.   4.5  0. ]]


[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
[ 1.   5.5 10. ]


In [13]:
optimized_Ndiff_kernel.fixed_or_free_paramdict

{'Ndiff_exponent': {'fixed_or_free': 'free',
  'const': 'free',
  'location_idx': (0, 1)},
 'p_bandscale': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (1, 3)},
 'Ndiff_depth_bw': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (3, 5)},
 'outer_x_bw': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (5, 6)},
 'outer_y_bw': {'fixed_or_free': 'free',
  'const': 'non-neg',
  'location_idx': (6, 7)},
 'free_params': array([ 0.00321122,  0.8469173 , -0.29390413,  0.35257914, -0.06770112,
        -0.00905423,  1.66152122]),
 'fixed_params': array([], dtype=float64)}

In [14]:
np.ma.count_masked(yhat)

0